# Grupo 10 ~ The N-queens Problem

**SSC0713 - Sistemas Evolutivos Aplicados à Robótica**

*André Baconcelo Prado Furlanetti - N°USP:* 

*Mateus Fernandes Doimo - N°USP: 10691971*

*Renan Peres Martins - N°USP: 10716612*

## Preparação do ambiente

Carrega as bibliotecas e inicia as constantes.

In [1]:
import numpy as np
from random import randrange, seed
import json

MAX_POPULATION = 50
MUTATION_RATE = 1
N_SIZE = 13

# Tabuleiro

*Define as funções auxiliares para manipular o tabuleiro*

* **initializeBoard** $\rightarrow$ Atribui um tabuleiro quadrado de tamanho N com todas as casa vazias;
* **insertPiece** $\rightarrow$ Insere uma rainha em dada coordenada;
* **removePiece** $\rightarrow$ Remove a peça de uma dada coordenada;
* **verifyAttack** $\rightarrow$ Verifica se há duas ou mais rainha se atacando e marca a quantidade de ataques realizados.


In [2]:
def initializeBoard():
    board = np.zeros([N_SIZE, N_SIZE], dtype=int)
    return board

def insertPiece(board, position_X, position_Y):
    if board[position_X][position_Y] != 0:
        return -1
    board[position_X][position_Y] = 1

    return board

def removePiece(board, position_X, position_Y):
    if board[position_X][position_Y] == 0:
        return -1
    board[position_X][position_Y] = 0

    return board

def verifyAttack(board, position_X, position_Y):
    atk = 0
    typeCheck = board[position_X][position_Y]

    if position_X >= N_SIZE or position_Y >= N_SIZE:
        return -1

    if typeCheck == 1:
        for i in range(N_SIZE):
            if i == position_Y:
                continue
            if board[position_X][i] != 0:
                atk += 1
        for i in range(N_SIZE):
            if i == position_X:
                continue
            if board[i][position_Y] != 0:
                atk += 1

    if typeCheck == 1:
        i = 1
        while (position_X + i < N_SIZE) and (position_Y + i < N_SIZE):
            if board[position_X + i][position_Y + i] != 0:
                atk += 1
            i += 1
        i = 1
        while (position_X - i >= 0) and (position_Y - i >= 0):
            if board[position_X - i][position_Y - i] != 0:
                atk += 1
            i += 1
        i = 1
        while (position_X + i < N_SIZE) and (position_Y - i >= 0):
            if board[position_X + i][position_Y - i] != 0:
                atk += 1
            i += 1
        i = 1
        while (position_X - i >= 0) and (position_Y + i < N_SIZE):
            if board[position_X - i][position_Y + i] != 0:
                atk += 1
            i += 1

    return atk, board

# População

*Define as funções auxiliares para manipular a população*

* **initializePopulation** $\rightarrow$ Atribui um conjunto de amostras de uma população de tamanho MAX_POPULATION x N;
* **mutation** $\rightarrow$ Realiza a mutação nas amostras de uma população em busca de uma evolução dos resultados do conjunto.

In [3]:
def initializePopulation():
    population = np.zeros([MAX_POPULATION, N_SIZE], dtype=int)
    for i in range(MAX_POPULATION):
        for j in range(N_SIZE):
            population[i][j] = randrange(start=0, stop=N_SIZE)
    return population

def mutation(population, mutationRate):
    for i in range(MAX_POPULATION):
        if((randrange(start=0, stop=1000)/1000.0) <= MUTATION_RATE):
            for j in range(mutationRate):
                genome = randrange(start=0, stop=N_SIZE)
                if(population[i][genome] == 0):
                    population[i][genome] += 1
                elif(population[i][genome] == N_SIZE-1):
                    population[i][genome] -= 1
                elif(randrange(start=0, stop=2)):
                    population[i][genome] += 1
                else:
                    population[i][genome] -= 1
    return population

# Amostras

*Define as funções auxiliares para manipular o uso das amostras de indivíduos*

* **insertSample** $\rightarrow$ Adiciona um conjunto de amostras no tabuleiro;
* **valuationSample** $\rightarrow$ Avalia o desempenho de um conjunto de indivíduos e retorna o Fitness dessas amostras.

In [4]:
def insertSample(sample, board):
    for i in range(N_SIZE):
        board[i][sample[i]] = 1
    return board

def valuationSample(board, sample):
    sumFitness = 0
    atk = 0
    board = initializeBoard()
    for i in range(N_SIZE):
        board = insertPiece(board, i, sample[i])
    for i in range(N_SIZE):
        atk, board = verifyAttack(board, i, sample[i])
        sumFitness += atk
        board = removePiece(board, i, sample[i])
    return sumFitness, board

# Fitness e Método adaptativo com Elitismo

*Define as funções auxiliares para gerar o vetor Fitness e realizar a recombinação das amostras pelo método*

* **generateFitnessVector** $\rightarrow$ Gera o vetor com os valores de Fitness de cada conjunto de amostras de uma população para uma dada geração;
* **recombinesSampleElitism** $\rightarrow$ Define o melhor Fitness de uma população naquela geração.

In [5]:
def generateFitnessVector(population, board):
    fitness_population = np.zeros(MAX_POPULATION, dtype=int)
    for i in range(MAX_POPULATION):
        fitness_population[i], board = valuationSample(board, population[i])
    return fitness_population, board
    
def recombinesSampleElitism(population, fitness):
    indexBetterFitness = 0
    betterFitness = 100000000

    for i in range(MAX_POPULATION):
        if(fitness[i] < betterFitness):
            indexBetterFitness = i
            betterFitness = fitness[i]
    for i in range(MAX_POPULATION):
        for j in range(N_SIZE):
            population[i][j] = population[indexBetterFitness][j]
    return indexBetterFitness, population


# Exibição da Simulação 

*Define as funções auxiliares para exibir o resultado obtido e gerar a animação de evolução das populações pelo algorítimo*

* **printBoard** $\rightarrow$ Printa o resultado no terminal (X são as rainhas e 0 as casas vazias);
* **generateFileBoard** $\rightarrow$ Gera um JSON para cada ordenações das rainhas no tabuleiro para as melhores amostras de uma dada geração;
* **generateFileJson** $\rightarrow$ Gera o arquivo JSON com todos os dados coletados por geração.

In [6]:
def printBoard(board):
    for i in range(N_SIZE):
        for j in range(N_SIZE):
            if board[i][j] == 1: print(" X ", end="")
            else: print(" 0 ", end="")
        print("")

def generateFileBoard(board, generation, stingJson, end, genocide, fitness):
    boardJson = '"'+str(generation) + \
        '" : { "genocidio": '+str(genocide) + \
        ', "fitness": '+str(fitness)+', "board": ['
    dictionary = ""
    for i in range(N_SIZE):
        for j in range(N_SIZE):
            if board[i][j] == 1:
                dictionary = dictionary+'{"row": '+str(i)+',"col": '+str(j)+'}'
                if i != (N_SIZE-1):
                    dictionary = dictionary+","
    boardJson = boardJson+dictionary+'] }'
    if end == 1:
        boardJson = boardJson+','
    stingJson = stingJson+boardJson
    return stingJson


def generateFileJson(stingJson):
    stingJson = stingJson+"}"
    json_object = json.loads(stingJson)
    with open("simulation.json", "w") as outfile:
        json.dump(json_object, outfile)

# Arquivo Main

*Função principal que repete os experimentos e testes com uma ou mais populações de amostras, por meio do método adaptivo com elitismo, buscando um Fitness zero*

In [7]:
#Variáveis locais
genocides = 0
stop_criterion = 1
generation = 0
mutation_rate = 1
fitness_previous = 1000000
stingJson = '{'

best_individual = np.zeros(N_SIZE, dtype=int)
board = initializeBoard()

#Laço de repetição
while stop_criterion:
        population = initializePopulation()
        if generation > 0:
            if randrange(start=0, stop=2):
                for i in range(N_SIZE):
                    population[0][i] = best_individual[i]
        while True:
            fitness, board = generateFitnessVector(population, board)
            index_best_individual, population = recombinesSampleElitism(population,fitness)
            for i in range(N_SIZE):
                best_individual[i] = population[index_best_individual][i]
            population = mutation(population, mutation_rate)
            board = initializeBoard()
            board = insertSample(population[index_best_individual], board)
            boardAnt = board
            generation += 1
            fitness_best_individual, board = valuationSample(
                board, best_individual)
            if fitness_best_individual == 0:
                board = insertSample(best_individual, board)
                stingJson = generateFileBoard(board, generation, stingJson, 0, 0, fitness_best_individual)
                generateFileJson(stingJson)
                printBoard(board)
                print("Generation: ", generation)
                print("genocidios: ", genocides)
                print("fitness: ", fitness_best_individual)
                stop_criterion = 0
                break
            if fitness_best_individual == fitness_previous:
                mutation_rate += 1
            elif fitness_best_individual < fitness_previous:
                mutation_rate = 0
            fitness_previous = fitness_best_individual
            if mutation_rate == N_SIZE:
                genocides += 1
                mutation_rate = 1
                population = 0
                stingJson = generateFileBoard(boardAnt, generation, stingJson, 1, 1, fitness_best_individual)
                break
            else:
                stingJson = generateFileBoard(boardAnt, generation, stingJson, 1, 0, fitness_best_individual)
print("End")


 0  0  0  0  0  0  0  0  0  0  0  X  0 
 0  0  0  0  X  0  0  0  0  0  0  0  0 
 0  0  0  0  0  0  0  X  0  0  0  0  0 
 0  X  0  0  0  0  0  0  0  0  0  0  0 
 0  0  0  X  0  0  0  0  0  0  0  0  0 
 0  0  0  0  0  0  0  0  0  0  0  0  X 
 0  0  0  0  0  0  X  0  0  0  0  0  0 
 0  0  0  0  0  0  0  0  0  X  0  0  0 
 X  0  0  0  0  0  0  0  0  0  0  0  0 
 0  0  0  0  0  0  0  0  0  0  X  0  0 
 0  0  0  0  0  X  0  0  0  0  0  0  0 
 0  0  X  0  0  0  0  0  0  0  0  0  0 
 0  0  0  0  0  0  0  0  X  0  0  0  0 
Generation:  760
genocidios:  0
fitness:  0
End
